# Easy to train different dictionaries on different hyperspectral images

In [1]:
import scipy.io as spio
import numpy as np
import random
import spams
from scipy import sparse
import matplotlib.pyplot as plt

In [2]:
IP_data_mat=spio.loadmat("data/IP/Indian_pines_corrected.mat")
IP_gt_mat=spio.loadmat("data/IP/Indian_pines_gt.mat")
IP_image=IP_data_mat["indian_pines_corrected"]
IP_gt=IP_gt_mat['indian_pines_gt']

salinas_mat = spio.loadmat('data/Salinas/salinas.mat')
salinas_gt_mat = spio.loadmat('data/Salinas/salinas_gt.mat')
salinas_image = salinas_mat['salinasA_corrected']
salinas_gt = salinas_gt_mat['salinasA_gt']

pavia_mat = spio.loadmat('data/PaviaU/PaviaU.mat')
pavia_gt_mat = spio.loadmat('data/PaviaU/PaviaU_gt.mat')
pavia_image = pavia_mat['paviaU']
pavia_gt = pavia_gt_mat['paviaU_gt']

In [3]:
def progress(curr_it, max_it):
    if (curr_it) < max_it:
        print('[{0}] {1}%'.format((curr_it+1), np.round(((curr_it+1)/max_it)*100,2)), end='\r')
    else:
        print('[{0}] {1}%'.format((curr_it+1), np.round(((curr_it+1)/max_it)*100,2)), end='\n')
        
# Dictionary update step from ODL, taken from J Mairal "Online Dictionary Learning for Sparse Coding"
def updateDict(D, A, B):
    D = D.copy()
    DA  = D.dot(A)
    count = 0
    for j in range(D.shape[1]):
        u_j = (B[:, j] - np.matmul(D, A[:, j]))/A[j, j] + D[:, j]
        D[:, j] = u_j/max([1, np.linalg.norm(u_j)])
    D = np.array(D, dtype=np.double)
    return D

class Joint_ODL:
    'Provide hyperspectral image and perform dictionary learning on it in a joint fashion'
    def drawRand(self):
        rand = random.randint(0,self.tiled.shape[0]-1)
        return self.tiled[:,rand]
    def initD(self):
        self.atoms = np.zeros((self.tiled.shape[0], self.num_coms))
        for i in range(self.num_coms):
            self.atoms[:,i]=self.drawRand()
    def normaliseD(self):
        D = self.atoms.copy()
        self.atoms = np.asfortranarray(D / np.tile(np.sqrt((D*D).sum(axis=0)),(D.shape[0],1)),dtype=np.double)
    def set_indices(self):
        self.ind_groups=np.array(np.arange(0,self.tiled.shape[1],self.tileDim*self.tileDim), dtype=np.int32)
    def tile(self):
        hsi = self.image.copy()
        ts = self.tileDim
        tiles = np.array([hsi[i:i+ts, j:j+ts]for j in range(0, hsi.shape[1], ts) for i in range(0, hsi.shape[0], ts)])
        tiles = np.reshape(tiles, (tiles.shape[0], ts*ts, tiles.shape[3]))
        tiles = np.reshape(tiles, (tiles.shape[0]*tiles.shape[1], tiles.shape[2]))
        return np.asfortranarray(tiles.T, dtype=np.double)
    def __init__(self, k, image, tilesize=10):
        if k <= image.shape[2]:
            print('Select an adequate number of components for %s.' %str(image.shape))
            return
        else:
            self.num_coms = k
        self.image = image
        self.tileDim = tilesize
        self.tiled = self.tile()
        self.ind_groups=None
        self.set_indices()
        self.atoms = None
        self.coefs = None
        self.initD()
        self.normaliseD()
    def get_coefs(self):
        self.coefs = spams.somp(self.tiled[:,:], self.atoms, self.ind_groups, L=self.L, eps=self.eps, numThreads=self.numThreads)
        return self.coefs
    def fit(self, max_iter=1000, L=3, eps=0.1, numThreads=-1):
        self.L = L
        self.eps = eps
        self.numThreads = numThreads
        A=np.zeros((self.num_coms, self.num_coms))
        B=np.zeros((self.tiled.shape[0], self.num_coms))
        for i in range(max_iter):
            progress(i, max_iter)
            random_index = random.choice(self.ind_groups)
            signals = self.tiled[:,random_index:(random_index+(self.tileDim**2))]
            self.set_indices()
            coefs = spams.somp(signals, self.atoms, self.ind_groups[:1], L=self.L, eps=self.eps, numThreads=self.numThreads)
            for j in range(coefs.shape[1]):
                signal = signals[:,j]
                coef = sparse.coo_matrix(coefs[:,j])
                alpha = coef.todense()
                alpha = np.array(alpha, dtype=np.double)
                alpha = alpha/np.sqrt(sum(alpha**2))
                #alpha = coefs[:,j]
                A += (np.dot(alpha.T, alpha)[0,0])
                B += (signal[:, None]*alpha[:].T)
                self.atoms = updateDict(self.atoms, A, B)
                self.atoms = np.asfortranarray(self.atoms)

In [4]:
pavia_jsrD = Joint_ODL(200, pavia_image, tilesize=10)
IP_jsrD = Joint_ODL(330, IP_image, tilesize=5)
pavia_jsrD.fit(max_iter=10)

ValueError: index pointer should start with 0

In [ ]:
pavia_dict = pavia_jsrD.atoms
print(pavia_dict)

In [ ]:
pavia_coefs = pavia_jsrD.get_coefs()
pavia_coefs = sparse.coo_matrix(pavia_coefs)
pavia_coefs = pavia_coefs.todense()

In [ ]:
print(pavia_coefs.shape)
pavia_pixels = np.reshape(pavia_image, (-1, 103)).T
print(pavia_pixels.shape)

In [ ]:
pavia_recon = np.dot(pavia_dict, pavia_coefs)
print(pavia_recon.shape)

In [ ]:
plt.plot(pavia_recon[:,20], label='Reconstruction')
plt.plot(pavia_pixels[:,20], label='Original')
plt.legend()
plt.show()